In [1]:
import re
import textdistance
from itertools import combinations

In [7]:
def open_file(name):
    fileObject = open(name, 'r', encoding='utf-8')
    posts_read = fileObject.read()
    return posts_read

# 1st group

In [17]:
fileObject = open('horoscopes.txt', 'r', encoding='utf-8')
posts_read = fileObject.read()

In [18]:
posts_read = posts_read.split('"\n')

In [19]:
pattern = re.compile('.+ ?\n ?\n')

In [20]:
re.sub(pattern, '', posts_read[0])

'Рыб нельзя заменить никем и никогда. \nЗапомните!'

In [21]:
bad_lst_none = []
bad_lst_many = []
for ind, line in enumerate(posts_read):
    find_pattern = re.findall(pattern, line)
    if find_pattern == []:
        bad_lst_none.append(ind)
    elif len(find_pattern) > 1:
        bad_lst_many.append(ind)

In [22]:
print(len(bad_lst_none))
print(len(bad_lst_many))

406
74


In [37]:
for ind in bad_lst_none:
    print(ind, posts_read[ind])

45 "Рыбы | 31 июля 
Весь день охеренно себя чувствуете, никакой спешки, всё степенно и зашибато. К вечеру флиртанёте с незнакомым человеком, амурные приключения.
389 "Рыбы | 31 июля
Весь день охеренно себя чувствуете, никакой спешки, всё степенно и зашибато. К вечеру флиртанёте с незнакомым человеком, амурные приключения.
447 "Рыбы - будьте осторожны, сегодня вы рискуете влюбится.
448 "Избегайте напряженных и рисковых ситуаций. Старайтесь не думать о политике и не смотреть новости. Отдохните, где-нибудь подальше от телевизора, радио и газет. Вам нужна полная перезагрузка системы и переосмысление ценностей.
449 "Рыбы - кто-то в вас влюбился и боится вам в этом признаться, вы его знаете. Возможно вам нужно ему немножечко помочь.
451 "Рыбы хотят, чтобы в их жизни был человек, которому можно сказать: А поехали за тульскими пряниками?. И вот, спустя несколько часов, ты стоишь в Туле, жуя пряник, и понимаешь, что жизнь прекрасна.
452 "Рыбам нужно немножечко алкоголя и хорошей вдохновляющей н

In [38]:
for ind in bad_lst_many:
    print(ind, posts_read[ind])

603 "РЫБЫ 

Гороскоп на 2018 год.В предстоящем году сможете оказаться в необычных местах, ситуациях и обстоятельствах, которые другим даже и не снились. Вероятно, решитесь на перемену места жительства или на долгие поездки. Но все же оставайтесь реалистичными, чтобы не гнаться за уходящей призрачной фата-морганой. Успех и личная популярность будут сопровождать личные начинания. Люди будут видеть ваши лучшие черты. Возможна важная встреча, внезапная популярность, прибыль. 
 
Появятся новые финансовые возможности, но они могут быть авантюрными. Январь подарит материальный успех, но далее придется много работать. В конце месяца последите за здоровьем. С 2018 года вас тянет в дорогу, где вы ищете воплощения высших идеалов. В поездках изменятся взгляды, реализуются некоторые мечты, а также появятся новые. Откроются новые таланты и способности, вы сможете положить начало многочисленным проектам. Некоторые останутся пустыми мечтаниями, а какие-то воплотятся в течение нескольких лет, неожиданн

In [23]:
good_lst = list(set(range(len(posts_read))) - set(bad_lst_many) - set(bad_lst_none))
posts_clear_1 = []

In [24]:
for ind in good_lst:
    posts_clear_1.append(re.sub(pattern, '', posts_read[ind]))

In [25]:
print(len(posts_clear_1))
print(len(set(posts_clear_1)))

31213
8959


In [26]:
posts_clear_2 = list(set(posts_clear_1))

In [28]:
fileObject = open('clear_horoscope_1_1.txt', 'w', encoding='utf-8')
fileObject.write('***'.join(posts_clear_2))

1135026

In [107]:
posts_splitted = []
for post in posts_clear_2:
    splitted = re.split(re.compile(r'[\n,. !?;]+'), post.lower())
    while '' in splitted:
        splitted.remove('')
    posts_splitted.append(splitted)

In [120]:
%%time
distances = []
for i, j in combinations(range(len(posts_splitted)), 2):
    dist = textdistance.jaccard(posts_splitted[i], posts_splitted[j])
    distances.append((i, j, dist))

Wall time: 47min 44s


In [122]:
%%time
distances = sorted(distances, key=lambda tup: tup[2], reverse=True)

Wall time: 33.4 s


In [124]:
len(distances)

40127361

In [182]:
mask = [True] * len(posts_clear_2)
for tup in distances[:11000]:
    i = tup[0]
    j = tup[1]
    if mask[i] & mask[j]:
        if len(posts_splitted[i]) > len(posts_splitted[j]):
            mask[j] = False
        mask[i] = False

In [186]:
posts_clear_3 = []
for i in range(len(posts_clear_2)):
    if mask[i]:
        posts_clear_3.append(posts_clear_2[i])

In [187]:
len(posts_clear_3)

4051

In [216]:
posts_splitted_2 = []
for post in posts_clear_3:
    splitted_2 = re.split(re.compile(r'[\n,. !?;]+'), post.lower())
    while '' in splitted_2:
        splitted_2.remove('')
    while '\u2000' in splitted_2:
        splitted_2.remove('\u2000')
    while '✨' in splitted_2:
        splitted_2.remove('✨')
    while '😎' in splitted_2:
        splitted_2.remove('😎')
    posts_splitted_2.append(splitted_2)

In [217]:
%%time
distances = []
for i, j in combinations(range(len(posts_splitted_2)), 2):
    dist = textdistance.jaccard(posts_splitted_2[i], posts_splitted_2[j])
    distances.append((i, j, dist))

Wall time: 14min 49s


In [218]:
%%time
distances = sorted(distances, key=lambda tup: tup[2], reverse=True)

Wall time: 6.67 s


In [231]:
distances[10]

(2409, 2462, 0.6428571428571429)

In [232]:
mask = [True] * len(posts_clear_2)
for tup in distances[:10]:
    i = tup[0]
    j = tup[1]
    if mask[i] & mask[j]:
        if len(posts_splitted_2[i]) > len(posts_splitted_2[j]):
            mask[j] = False
        mask[i] = False

In [235]:
tmp = posts_clear_3
posts_clear_3 = []
for i in range(len(tmp)):
    if mask[i]:
        posts_clear_3.append(tmp[i])

In [240]:
for i in range(len(posts_clear_3)):
    posts_clear_3[i] = posts_clear_3[i].replace('😎', '')
    posts_clear_3[i] = posts_clear_3[i].replace('✨', '')
    posts_clear_3[i] = posts_clear_3[i].replace('\u2000', '')    

In [245]:
fileObject = open('clear_horoscope_1.txt', 'w', encoding='utf-8')
fileObject.write('***'.join(posts_clear_3))

521695

In [246]:
len(posts_clear_3)

4040